In [1]:
from os import listdir
from os.path import join, isfile
from lxml import etree
from src.data import PackageManager
from src.data import XbrlConst
from collections import defaultdict
from src.data import xbrl2rdf, process_linkbase, process_instance
import src.data
from io import StringIO, BytesIO
import rdflib

In [2]:
XBRL_TAXONOMY_PATH = join('..', 'data', 'external', 'taxonomies')
output_dir = join("..", "data", "raw")

taxonomies = [f for f in listdir(XBRL_TAXONOMY_PATH) if isfile(join(XBRL_TAXONOMY_PATH, f)) and f[-3:]=='zip']

manager = PackageManager.Taxonomies(XBRL_TAXONOMY_PATH)

for taxonomy in taxonomies:
    manager.addPackage(join(XBRL_TAXONOMY_PATH, taxonomy))
manager.rebuildRemappings()
manager.save()
for item in manager.config['packages']:
    print(item['URL'])

Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Pack

In [3]:
f = src.data.openFileSource(manager.config['packages'][2]['URL'])
f.mappedPaths = manager.config['packages'][1]["remappings"]

In [4]:
xmlfiles = [file for file in f.dir if (file[-3:]=='xml')]
print(len(xmlfiles))

30603


In [5]:
roots = list()

for file in xmlfiles:
    fp = f.fs.open(file, "r")
    content = fp.read()
    roots.append(etree.fromstring(content, parser=etree.XMLParser(remove_comments=True)))

In [6]:
namespaces = {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
              'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
              'xl': 'http://www.xbrl.org/2003/XLink',
              'xsd': 'http://www.w3.org/2001/XMLSchema',
              'ext': 'http://eiopa.europa.eu/eu/xbrl/ext',
              'eurofiling_arcrole': 'http://www.eurofiling.info/xbrl/arcrole',
              'arcrole_2008': 'http://xbrl.org/arcrole/2008',
              'arcrole_2010': 'http://xbrl.org/arcrole/2010',
              'arcrole_2003': 'http://www.xbrl.org/2003/arcrole',
              'role_2003': 'http://www.xbrl.org/2003/role',
              'role_2008': 'http://www.xbrl.org/2008/role',
              'role_2010': 'http://www.xbrl.org/2010/role',
              'arcrole_dim': 'http://xbrl.org/int/dim/arcrole'}
for idx, root in enumerate(roots):
    namespaces = {**namespaces, **root.nsmap}
if None in namespaces.keys():
    del namespaces[None]

In [7]:
params = {'linkNumber' : 0, 
          'namespaces': namespaces}

prefix = '# RDF triples (in turtle syntax) imported from XBRL resource at URI\n'
prefix += '# '+manager.config['packages'][2]['URL']+'\n\n'
prefix += '# the namespaces\n'
for namespace in namespaces.keys():
    prefix += "@prefix "+namespace+": <"+namespaces[namespace]+">.\n"
prefix += '\n'

s = prefix
g = rdflib.Graph()
for idx, root in enumerate(roots[0:100]):
    params['output'] = StringIO()
    params['base'] = xmlfiles[idx]
    params = process_linkbase.processLinkBase(root, params)
    string = params['output'].getvalue().replace('\u2264', '')
    s += string
    g.parse(data=prefix+string, format='turtle')

In [ ]:
fp = open(join(output_dir, "taxo.ttl"), "w", encoding = "utf-8")
fp.write(s)
fp.close()

In [ ]:
g.serialize(destination=join(output_dir,'output.txt'), format='turtle')

In [ ]:
# print the number of "triples" in the Graph
print("graph has {} statements.".format(len(g)))